In [1]:
!pip install scrapy

In [2]:
import scrapy
from scrapy.crawler import CrawlerProcess
import re
import string

In [3]:
class GoodreadQuotesSpider(scrapy.Spider):
    name = "quotes_spider"
    start_urls = [f"https://www.goodreads.com/quotes?page={p}" for p in range(1, 10)]

    custom_settings = {
        'FEEDS': {
            'quotes.csv': {
                'format': 'csv',
                'overwrite': True
            }
        }
    }

    def parse(self, response):
        SET_SELECTOR = '.quoteText'
        for brickset in response.css(SET_SELECTOR):

            DETAILS_SELECTOR = 'div ::text'
            AUTHOR_SELECTOR = '.authorOrTitle ::text'
            yield {
                'details': brickset.css(DETAILS_SELECTOR).extract_first(),
                'author': brickset.css(AUTHOR_SELECTOR).extract_first(),
            }







process = CrawlerProcess()
process.crawl(GoodreadQuotesSpider)
process.start()


INFO:scrapy.utils.log:Scrapy 2.9.0 started (bot: scrapybot)
2023-08-01 13:04:51 [scrapy.utils.log] INFO: Scrapy 2.9.0 started (bot: scrapybot)
INFO:scrapy.utils.log:Versions: lxml 4.9.3.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.1, Twisted 22.10.0, Python 3.10.6 (main, May 29 2023, 11:10:38) [GCC 11.3.0], pyOpenSSL 23.2.0 (OpenSSL 3.1.1 30 May 2023), cryptography 41.0.2, Platform Linux-5.15.109+-x86_64-with-glibc2.35
2023-08-01 13:04:51 [scrapy.utils.log] INFO: Versions: lxml 4.9.3.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.1, Twisted 22.10.0, Python 3.10.6 (main, May 29 2023, 11:10:38) [GCC 11.3.0], pyOpenSSL 23.2.0 (OpenSSL 3.1.1 30 May 2023), cryptography 41.0.2, Platform Linux-5.15.109+-x86_64-with-glibc2.35
INFO:scrapy.crawler:Overridden settings:
{}
2023-08-01 13:04:51 [scrapy.crawler] INFO: Overridden settings:
{}
/usr/local/lib/python3.10/dist-packages/scrapy/utils/request.py:232: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'R

In [4]:
stopwords= ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're" , "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'th ey', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "tha t'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'ha d', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'ove r', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any' , 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'no w', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'might n', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wa sn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"]

In [5]:
import pandas as pd




def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

def preprocess_text(sentance):
    sent = decontracted(sentance)
    sent = re.sub(r'[^\x00-\x7F]+', ' ', sentance)
    sent = re.sub(r'@\w+', '', sent)
    sent = sent.lower()
    sent = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', sent)
    sent = re.sub(r'[0-9]', '', sent)
    sent = re.sub(r'\s{2,}', ' ', sent)
    # sent = sent.replace('\\r', ' ')
    # sent = sent.replace('\\"', ' ')
    # sent = sent.replace('\\n', ' ')
    # https://gist.github.com/sebleier/554280
    sent = ' '.join(e for e in sent.split() if e not in stopwords)
    return sent

quotes = pd.read_csv('quotes.csv', encoding='latin')
quotes['normal'] = quotes['details'].map(lambda x: preprocess_text(x))

quotes.head(10)

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
2023-08-01 13:04:57 [numexpr.utils] INFO: NumExpr defaulting to 2 threads.


,details,author,normal
0,\n âBe yourself; everyone else is alrea...,\n Oscar Wilde\n,everyone else already taken
1,"\n âI'm selfish, impatient and a little...",\n Marilyn Monroe\n,selfish impatient little insecure make mistake...
2,\n âTwo things are infinite: the univer...,\n Albert Einstein\n,two things infinite universe human stupidity n...
3,"\n âSometimes, you read a book and it f...","\n John Green,\n",sometimes read book fills weird evangelical ze...
4,"\n âSo many books, so little time.â\n...",\n Frank Zappa\n,many books little time
5,\n âA room without books is like a body...,\n Marcus Tullius Cicero\n,room without books like body without soul
6,\n âI wish it need not have happened in...,"\n J.R.R. Tolkien,\n",wish need not happened time said frodo
7,\n âAll you need is love. But a little ...,\n Charles M. Schulz\n,need love little chocolate now hurt
8,\n âLife isn't about finding yourself. ...,\n George Bernard Shaw\n,life finding life creating
9,\n âBe who you are and say what you fee...,\n Bernard M. Baruch\n,say feel mind matter matter mind


In [6]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.feature_extraction.text import CountVectorizer

In [7]:
vectorizer = TfidfVectorizer()
bow = vectorizer.fit_transform(quotes["normal"].astype('U'))
print("Shape of train matrix after BOW : ",bow.shape)


Shape of train matrix after BOW :  (270, 1175)


In [8]:
matrix = pd.DataFrame(bow.T.toarray(), index=vectorizer.get_feature_names())
matrix.head()

AttributeError: ignored

In [9]:
import numpy as np

def search(q, df):
  print("query:", q)
  q = [q]
  q_vec = vectorizer.transform(q).toarray().reshape(df.shape[0],)
  sim = {}
  for i in range(df.shape[1]):
    sim[i] = np.dot(df.loc[:, i].values, q_vec) / np.linalg.norm(df.loc[:, i]) * np.linalg.norm(q_vec)

  sim_sorted = sorted(sim.items(), key=lambda x: x[1], reverse=True)
  for k, v in sim_sorted[:10]:
    if v != 0.0:
      print("result:", v)
      print("------------")
      print(quotes["details"][k])
      print(quotes["normal"][k])
      print("by : '", quotes["author"][k], "'")
      print("------------")
      print()

In [ ]:
  query=input()
  normal_query=preprocess_text(query)
  search(normal_query, matrix)